# Notebook #1: Captura y Limpieza de Datos

En este notebook realizaremos el webscraping y la limpieza de los datos que vamos a cargar en nuestra base de datos.

El primer paso será importar las librerías necesarias:

In [2]:
# Librerías para tratamiento de datos

import pandas as pd
pd.set_option('display.max_columns', None) # Parámetro que modifica la visualización de los DFs
import numpy as np

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.
from src import soporte_variables as sv

# Librería para exportar archivos de tipo pickle
import pickle

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

### Paso 1: Obtener Links de Productos
- Dado que hemos creado todas nuestras funciones en `soporte_funciones.py`, dentro de la carpeta `/src`, en este notebook sólamente hará falta llamar a las funciones.

- La función `sf.captura_links_facua`, recibe como parámetro el link de Faqua donde se pueden consultar precios históricos del aceite de girasol, el aceite de oliva y leche, y devuelve una lista con los links de esos productos para los 6 supermercados que pueden consultarse. Para obtenerlos, utiliza Selenium y Beautiful Soup.

- Dado que los datos se modifican cada vez que se ejecuta una función, las mismas están comentadas. Para ejecutarlas, debe eliminarse la #.

In [3]:
url_faqua = "https://super.facua.org"
#lista_links = sf.captura_links_facua(url_faqua)

Click en cookies


  0%|          | 0/6 [00:00<?, ?it/s]

Click en supermercado: 1
Click en categoría: 1
Click en categoría: 2
Click en categoría: 3


 17%|█▋        | 1/6 [00:15<01:17, 15.46s/it]

Click en supermercado: 2
Click en categoría: 1
Click en categoría: 2
Click en categoría: 3


...

 67%|██████▋   | 4/6 [01:03<00:31, 15.97s/it]

Click en supermercado: 5
Click en categoría: 1
Click en categoría: 2
Click en categoría: 3


 83%|████████▎ | 5/6 [01:28<00:19, 19.13s/it]

Click en supermercado: 6
Click en categoría: 1
Click en categoría: 2
Click en categoría: 3


100%|██████████| 6/6 [01:44<00:00, 17.39s/it]


- Guardamos los links extraídos en un archivo de formato pickle, de modo que puedan utilizarse en otro momento sin necesidad de ejecutar nuevamente la captura. También los hemos guardado en un archivo .py.

In [4]:
#with open("../datos/links.pkl", 'wb') as archivo:
#    pickle.dump(lista_links, archivo)

- Comprobamos la longitud de la lista para saber cuántos links se han capturado en total. Esto es lo mismo que, la cantidad de productos de la que extraeremos el histórico. En el caso de la última consulta, con fecha 27/10/2024, hemos conseguido extraer 1665.

In [5]:
#len(lista_links)

1665

### Paso 2: Obtener Históricos de Productos

- Para cada uno de los productos, tenemos en su respectiva página una tabla que registra los precios históricos.

- Con la función `sf.captura_historicos_facua()`, entraremos a cada uno de los links obtenidos en el paso 1 (parámetro de la función), y utilizando BeautifulSoup extraeremos la tabla con los precios históricos.

- El resultado final es un DataFrame con los históricos de todos los productos para los 6 supermercados, a la que se le ha ejecutado un proceso de limpieza y transformación haciendo uso de Pandas.

In [3]:
# En caso de que nos interesara ejecutar esta segunda función sin haber ejecutado el paso 1, debemos cargar el archivo pickle que contiene los links, haciendo uso del siguiente código:

with open("../datos/links.pkl", 'rb') as archivo:
    lista_links = pickle.load(archivo)

In [6]:
df_completo = sf.captura_historicos_facua(lista_links)

  0%|          | 0/1665 [00:00<?, ?it/s]

100%|██████████| 1665/1665 [04:32<00:00,  6.12it/s]


- Ejecutada la captura de los históricos, imprimimos una muestra para comprobar que los datos sean los esperados y tengan el formato esperado, y contamos la cantidad de filas, para ver que tenemos unos 104 mil registros.

In [8]:
df_completo.sample(3)

,Fecha,Producto,Precio (€),Categoría,Supermercado,Var. Euros,Var. Porcentaje
71,16/09/2024,Aceite De Oliva Virgen Extra Hojiblanca Garraf...,56.66,Aceite-de-oliva,Eroski,=,None
112,12/10/2024,Aceite De Oliva Virgen Hacendado Botella 1 L,8.06,Aceite-de-oliva,Mercadona,=,None
29,13/08/2024,Puleva Leche Semidesnatada De Vacas Con Alimen...,7.44,Leche,Alcampo,=,None


In [9]:
df_completo.shape

(104375, 7)

- Revisamos también si tenemos datos duplicados, y vemos que hay 1724 duplicados.
- Se ha identificado que hay productos que aparecen duplicados en la página de Facua, existiendo dos entradas con la misma URL. Por este motivo, los eliminaremos del DF.

In [10]:
df_completo.duplicated().value_counts()

False    102651
True       1724
Name: count, dtype: int64

In [11]:
df_completo.drop_duplicates(inplace=True)

In [12]:
df_completo.shape

(102651, 7)

- Realizada la comprobación, nos quedamos con 102650 registros. Guardamos los datos en un archivo de tipo CSV, de modo que puedan utilizarse nuevamente sin ejecutar la consulta.

In [13]:
df_completo.to_csv("../datos/df_final_captura.csv")

### Paso 3: Preparar DFs para carga a SQL

- Tendremos una distribución de 4 tablas: supermercado, categoría, productos e histórico.

- Crearemos primero un DF de supermercados, con un ID que hemos asignado manualmente. En todos los casos, guardaremos estos DFs en archivos CSV.

In [14]:
diccionario_supermercados = {1: "Mercadona", 2: "Carrefour", 3: "Eroski", 4: "Dia", 5: "Hipercor", 6: "Alcampo"}
df_supermercados = pd.DataFrame(list(diccionario_supermercados.items()), columns=['ID Supermercado', 'Nombre'])
df_supermercados

,ID Supermercado,Nombre
0,1,Mercadona
1,2,Carrefour
2,3,Eroski
3,4,Dia
4,5,Hipercor
5,6,Alcampo


In [14]:
# df_supermercados.to_csv("../datos/df_supermercados.csv")

- Haremos lo mismo para las categorías, con un ID manual.

In [15]:
diccionario_categorias = {"ao": "Aceite-de-oliva", "ag": "Aceite-de-girasol", "le": "Leche"}
df_categorias = pd.DataFrame(list(diccionario_categorias.items()), columns=['ID Categoría', 'Nombre Categoría'])
df_categorias

,ID Categoría,Nombre Categoría
0,ao,Aceite-de-oliva
1,ag,Aceite-de-girasol
2,le,Leche


In [16]:
# df_categorias.to_csv("../datos/df_categorias.csv")

- Ahora crearemos uno para los productos, donde copiaremos la columa "Producto" de df_completo y eliminaremos los valores duplicados.

In [16]:
df_productos = df_completo[["Producto"]]
df_productos.drop_duplicates(inplace=True)

In [17]:
df_productos.shape

(1098, 1)

- Nos hemos quedado con unos 1100 productos únicos. Resetearemos los índices de modo que, se asigne un "ID Producto" de manera secuencial.

In [18]:
df_productos.reset_index(inplace=True)
df_productos.drop(columns = "index", inplace=True)
df_productos.reset_index(inplace=True)
df_productos = df_productos.rename(columns={"index": "ID Producto"})

In [19]:
df_productos.head(5)

,ID Producto,Producto
0,0,Aceite De Girasol Refinado 02 Hacendado 1 L
1,1,Aceite De Girasol Refinado 02 Hacendado 5 L
2,2,Aceite De Oliva 04 Hacendado
3,3,Aceite De Oliva 1 Hacendado Botella 1 L
4,4,Aceite De Oliva Intenso Hacendado Garrafa 3 L


In [47]:
# df_productos.to_csv("../datos/df_productos.csv")

- Vamos a crear ahora nuestro DF histórico, cuyas columnas serán: fecha, ID categoría, ID producto, ID Supermercado, Var. Euros y Variación Porcentaje.

- El primer paso será concatenar nuestro df_completo (obtenido en el Paso 2), con los 3 nuevos DFs que hemos creado hasta ahora (supermercado, categoría y producto). Eliminaremos las columnas que aporten información repetida.

In [20]:
df_completo = df_completo.merge(df_supermercados, left_on="Supermercado", right_on="Nombre")
df_completo = df_completo.merge(df_productos, left_on="Producto", right_on="Producto")
df_completo = df_completo.merge(df_categorias, left_on="Categoría", right_on="Nombre Categoría")
df_completo.drop(columns = "Nombre", inplace=True)
df_completo.drop(columns = "Nombre Categoría", inplace=True)

- Y ahora crearemos el nuevo DF, con las columnas que nos interesa mantener.

In [25]:
df_historico = df_completo[["Fecha", "ID Categoría","ID Producto","ID Supermercado","Precio (€)","Var. Euros","Var. Porcentaje"]]
df_historico.sample(5)

,Fecha,ID Categoría,ID Producto,ID Supermercado,Precio (€),Var. Euros,Var. Porcentaje
26120,20/08/2024,ao,282,3,38.95,"-2,00","(4,88%)"
52009,26/07/2024,ao,551,5,16.71,=,None
32535,03/10/2024,le,352,3,2.44,=,None
51680,05/09/2024,ao,547,5,13.29,=,None
49196,04/08/2024,ao,521,5,28.54,=,None


- Realizaremos algunos pasos adicionales de limpieza para que los datos estén preparados para la carga en SQL.

In [26]:
# Tratar formatos de números
df_historico["Var. Euros"] = (df_historico["Var. Euros"]
    .str.replace(",", ".")
    .str.replace("+", "")
    .str.replace("=", "0")
).astype(float)

# Tratar formatos de fecha
df_historico["Fecha"] = pd.to_datetime(df_historico["Fecha"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

- Calcularemos la columna de Variación Porcentaje nuevamente dado que, hemos identificado que habíamos perdido la condición de positivo o negativo.

In [27]:
df_historico["Var. Porcentaje"] = round((df_historico["Var. Euros"]/(df_historico["Precio (€)"]-df_historico["Var. Euros"]))*100,2)

- Tras estos pasos, nuestro "df_histórico" tendrá este aspecto:

In [30]:
df_historico.sample(3)

,Fecha,ID Categoría,ID Producto,ID Supermercado,Precio (€),Var. Euros,Var. Porcentaje
65569,2024-09-10,le,704,5,1.25,0.0,0.0
73369,2024-07-25,le,800,5,1.26,0.0,0.0
3706,2024-07-15,le,33,1,1.53,0.0,0.0


In [31]:
#df_historico.to_csv("../datos/df_historico.csv")

- Al final del tratamiento, tendremos también un "df_completo" con este formato:

In [33]:
df_completo.sample(2)

,Fecha,Producto,Precio (€),Categoría,Supermercado,Var. Euros,Var. Porcentaje,ID Supermercado,ID Producto,ID Categoría
31118,17/08/2024,Leche Entera Sin Lactosa Asturiana Brik 1 Litro,1.19,Leche,Eroski,=,None,3,337,le
85910,06/08/2024,Producto Alcampo Aceite De Oliva Intenso Botel...,7.60,Aceite-de-oliva,Alcampo,=,None,6,937,ao


- En el siguiente notebook -número 2-, crearemos la base de datos, las tablas y cargaremos los datos recogidos.